# Molecular dating of Holocephali using fossil calibrations


## 1. Introduction

### 1.1 L'article
Grâce à la leur position à la base des poissons __osseux__, les poissons __cartilagineux__ sont considérés comme un excellent _outgroup_ dans les études comparatives des vertebrés.
<br/>
Récemment, le génome de _Callorhinchus milii_ (__chimère éléphant__) a été proposé comme génome modèle pour comprendre l'histoire évolutive des génomes vertebrés grâce sa position phylogénétique et à son génome de petite taille (910 Mb). Malgré un tel intérêt croissant, l’histoire évolutive des _holophaceans_ modernes ainsi que leur rélation avec _Elasmobranchii_ (sous-classe de _Chondrichthyes_, qui regroupe les requins, raies et chimères) et d'autres vertebrés à machoire n'a pas été bien documentée à cause de l'absence de matériaux fossiles bien conservés.
<br/><br/>
Dans cette étude, les auteurs ont assemblé le génome mitchondrial de 8 représentants de trois familles d'_holophaceans_ modernes, pour pouvoir étudier leurs rélations phylogénétiques et histoire évolutive. Pour faire cela, ils ont analysé par __maximum__ de __vraisemblance__ les séquences alignées sans ambiguïté de ces _holophaceans_ avec 17 autres vertebrés (9,409 positions nucléotidiques en excludant la troisième position du codon).
<br/>
L'arbre généré supporte l'hypothèse d'une seule origine des _holophaceans_ modernes et leur relation avec le groupe-soeur des _Elasmobranchii_. 
L’arbre mitogénomique a récupéré le plus grand nombre de _Callorhinchus_ basaux dans les _Chimaeriformes_, lequel est groupe-soeur avec les clades des deux familles restantes (_Rhinochimaeridae_ et _Chimaeridae_). 
<br/>
L’arbre temporel dérivé d’une horloge moléculaire détendue bayésienne suggère que les _holophaceans_ sont originaires du __Silurien__ vers 420 Ma, ayant survécu dans la fin du __Permien__ à l'extinction de masse et qui ont subi une diversification familiale de la fin du __Jurassique__ au début du __Crétacé__. Ce scénario évolutif proposé est bien en accord avec celui basé sur les observations paléontologiques.
<br/><br/>
Source: [Evolutionary Origin and Phylogeny of the Modern Holocephalans (Chondrichthyes: Chimaeriformes): A Mitogenomic Perspective](https://academic.oup.com/mbe/article/27/11/2576/1121974?login=true)

### 1.2 Le projet
La datation d’une phylogénie repose sur l’interprétation de la divergence entre les séquences à la lumière des calibrations fossiles, et à la lumière des _a priori_ sur ces calibrations et sur la forme de la phylogénie. <br/>
Il s’agit d’un problème inférentiel difficile où les _a priori_ et les __mouvements__ qui sont utilisés peuvent avoir un impact important sur le résultat et sur la convergence du __MCMC__. Dans ce projet, on va ré-analyser un ensemble de données construit par _Inoue et al._ (cfr. __1.1 L'article__) pour évaluer la robustesse de leurs résultats par rapport aux _a priori_ ou aux __mouvements__ qui sont utilisés.

### 1.3 Les données
Comme anticipé dans la section __1.1 L'article__, les données sont constituées par des génomes mitochrondiaux entiers assemblés de 8 _holophaceans_ (répresentant les trois familles):
- _Callorhinchus milii_ (**_Callorhinchidae_**)
- _Callorhinchus capensis_ (**_Callorhinchidae_**)
- _Callorhinchus callorynchus_ (**_Callorhinchidae_**)
- _Hariotta raleighana_ (**_Rhinochimaeridae_**)
- _Rhinochimaera pacifica_ (**_Rhinochimaeridae_**)
- _Hydrolagus lemures_ (**_Chimaeridae_**)
- _Chimaera sp_ (**_Chimaeridae_**)
- _Chimaera monstrosa_ (**_Chimaeridae_**)

Et 15 autres vertebres: _Cyprinus carpio_, _Polymixia japonica_, _Tetraodon nigroviridis_, _Protopterus aethiopicus_, _Xenopus tropicalis_, _Struthio camelus_, _Ornithorhynchus anatinus_, _Homo sapiens_, _Plesiobatis daviesi_, _Okamejei kenojei_, _Amblyraja radiata_, _Squalus acanthias_, _Heterodontus francisci_, _Scyliorhinus canicula_, _Mustelus manazo_.
<br/>
Les séquences sont contenues dans le fichier `aln_prot.fst`. Il contient les séquences d'acides aminés de 12 gènes codant des protéines des génomes mitochondriaux (exclusion des gènes ARNt et ARNr).

In [1]:
# read nucleotide data
data <- readDiscreteCharacterData("../Projet/fishes/aln_prot.fst")

n_species <- data.ntaxa()
n_branches <- 2 * n_species - 2 # car on veut un arbre enraciné
taxa <- data.taxa()

# root age
root_age ~ dnUniform(0,1000)

   Missing Variable:	Variable os does not exist

   Successfully read one character matrix from file '../Projet/fishes/aln_prot.fst'


In [2]:
# tree is produced by a birth death process

# speciation rate
lambda <- 1.000000000000001 # it can be egal to mu, math error

# extinction rate
mu <- 1 


# sampling fraction (in the present)
# here, we assume that all species (all primates in this case) have been sampled
rho <- 0.00046 # 23 sampled species / 50'000 vertebrates species estimated

# time tree produced by a birth-death process (BDP)
timetree ~ dnBDP(lambda, mu, rho, root_age, samplingStrategy="uniform", condition="nTaxa", taxa=taxa)

In [3]:
# read tree from file
tree <- readTrees("../Projet/fishes/vertebrate_Poisson_NJ.tree", treetype="clock")[1]
# ajust terminal branch lengths so as to make the tree ultrametric
tree.makeUltrametric()

# set the value of the tree topology
timetree.setValue(tree)

# rescale root age to 1.0
# (all ages will be relative to the root)
# root_age.setValue(10.0) # fix to 10.0

   Attempting to read the contents of file "vertebrate_Poisson_NJ.tree"
   Successfully read file


In [11]:
clade_1 <- clade("Plesiobatis_daviesi","Mustelus_manazo")
speciation_clade_1 := tmrca(timetree, clade_1)
min_clade_1 <- 190.0
max_clade_1 <- 463.0
width_age_1 <- (max_clade_1 - min_clade_1) / 2.0
mean_age_1 <- min_clade_1 + width_age_1
obs_age_1 ~ dnNormal(speciation_clade_1, mean_age_1) #dnUniform(speciation_clade_1 - width_age_1, speciation_clade_1 + width_age_1)

# obs_age_1 ~ dnExponential(0.2, offset = - speciation_clade_1)

# ?moves sur obs_age
obs_age_1.clamp(-mean_age_1)
speciation_clade_1
mean_age_1

   0.23519
   326.5


In [ ]:
# we assume a relaxed molecular clock, of unknown rate
mean_clockrate ~ dnExponential(1.0)
relvar_clockrate ~ dnExponential(1.0)

alpha := 1.0 / relvar_clockrate
beta := alpha / mean_clockrate

for (i in 1:n_branches) {
    clockrate[i] ~ dnGamma(alpha, beta)
}

Q := fnMtRev()

seq ~ dnPhyloCTMC( tree=timetree, Q=Q, branchRates=clockrate, type="AA" )
# le taux d'évolution dans toutes les branches est égale au clockrate
seq.clamp( data )

my_model = model(timetree)

# moves
moves = VectorMoves()

# moves.append(mvScale(lambda, weight=1.0))
# moves.append(mvScale(mu, weight=1.0))

for (i in 1:n_branches) {
    moves.append(mvScale(clockrate[i], weight=1.0, lambda=0.5))    
}
moves.append(mvScale(relvar_clockrate, weight=1.0, lambda=0.1))
moves.append(mvScale(mean_clockrate, weight=1.0, lambda=0.1))

# moves.append(mvSlide(gamma, weight=1.0, delta=0.1))
# moves.append(mvScale(kappa, weight=1.0))
# moves.append(mvScale(clockrate, weight=1.0, lambda=0.1))

moves.append(mvScale(root_age, weight=1.0, lambda=0.1))
moves.append(mvNodeTimeSlideUniform(timetree, weight=20)) # bouge un seul noeud
moves.append(mvSubtreeScale(timetree, weight=4)) # bouge de façon proportionnel tous les noeuds d'un sous-arbre

monitors[1] = mnModel(filename="../Projet/analyses/fish_clock_1.log", printgen=10, separator = TAB)
monitors[2] = mnFile(timetree ,filename="../Projet/analyses/fish_clock_1.trees", printgen=10, separator = TAB)
monitors[3] = mnScreen(printgen=100, mean_clockrate, relvar_clockrate, root_age, obs_age_1)

# Make the analysis object
analysis = mcmc(my_model, monitors, moves)
# Run the MCMC
analysis.run(1000)

treetrace = readTreeTrace("../Projet/analyses/fish_clock_1.trees", treetype="clock", burnin=0.1)
map_tree = mccTree(treetrace, "../Projet/analyses/fish_clock_1.tree")

   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could n

   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   Could n